<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br><br><br><br><br>
<u><b><font size = 6>A2: Team Project OJ </f></b></u><br><br>
<b><font size = 5> Data Optimization - DAT-5304 </f></b>
<br><br>
<b>Authors</b>          : Carolina Novello Moreira, Jack Daoud & Max Lembke <br>
<b>Date Created</b> : 03/24/2021<br>
<br><br><br><br><br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br>


<b><font size = 5> Instructions </f></b>

This option involves using one of the two real-world datasets that I posted on Canvas. These datasets have information on Unit Sales, Prices, and Gross Margins for several SKU’s (Stock Keeping Units). The OJ Data is from Dominik’s Grocery Store in Chicago. The Milk Dataset is from Peru. Both have weekly data for multiple years. Using this data, you should advise management as to what they should do. Your advice could include marketing strategies (feature and display in OJ Dataset or Loyalty Card usage in Milk Dataset). Some suggestions or ideas are to: 

1.	Estimate Quantity Demanded as a function of price (and similar prices) using a Log Transformation. You can also include marketing strategies in your regression.
2.	Combine Gross Margin information to advise the firm on maximizing profits. To avoid sales cannibalization, simultaneously include multiple products. Depending on how you set this up, you might be able to use Solver in Excel to fin the optimal strategy. It is a NLP.
3.	The estimates of margins and demand are only estimates. Perform Monte Carlo analysis to simulate different strategies even if your estimate is not perfect (draw the regression coefficients from a normal with mean = estimate and standard deviation = standard error). Report on the results from these simulations.

# Set-up 

In [17]:
# import packages 
import pandas as pd #pandas 
import numpy  as np #numpy 
from numpy import arange # for ranges with floats
from pulp import * #pulp everything 
import statsmodels.formula.api as smf # predictive modeling with nice outputs
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from regressors import stats

# set options 
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 50)

# importing file 
df = pd.read_excel('OJ_Data.xlsx')

In [18]:
# helper functions 

# make binary function 
def make_binary(df):
    
    for col in df.columns:
        
        if 'feat' in col \
        or 'disp' in col:
            
            for index, value in df.iterrows():
                
                if df.loc[index, col] >= 0.5:
                    df.loc[index, col] = 1
                else:
                    df.loc[index, col] = 0
        
            df[col] = df[col].astype(int)

# rev and profit function 
def get_rev_and_profit(df):

    # create Revenue columns for each SKU
    for i in range(1,6):
        df['revenue' + str(i)] = 0
     
    
    # compute Revenue using price x sales
    for index, value in df.iterrows():
        for i in range(1,6):
            df.loc[index, 'revenue' + str(i)] \
            = df.loc[index, 'price' + str(i)] * df.loc[index, 'sales' + str(i)]
            
            df['revenue'+str(i)] = df['revenue' + str(i)].round(2)
     
    
    # create Profit columns for each SKU
    for i in range(1,6):
        df['profit' + str(i)] = 0
     
    
    # compute Profit using revenue x gross margin
    for index, value in df.iterrows():
        for i in range(1,6):
            df.loc[index, 'profit' + str(i)] = \
            df.loc[index, 'revenue' + str(i)] * df.loc[index, 'grmar' + str(i)]
            
            df['profit' + str(i)] = df['profit' + str(i)].round(2)

In [19]:
# data cleaning 

# calling make binary function 
make_binary(df)

# calc. rev and profits function 
get_rev_and_profit(df)


# log transformations 

# price
df['log_price1'] = np.log10(df['price1'])
df['log_price2'] = np.log10(df['price2'])
df['log_price3'] = np.log10(df['price3'])
df['log_price4'] = np.log10(df['price4'])
df['log_price5'] = np.log10(df['price5'])

# sales
df['log_sales1'] = np.log10(df['sales1'])
df['log_sales2'] = np.log10(df['sales2'])
df['log_sales3'] = np.log10(df['sales3'])
df['log_sales4'] = np.log10(df['sales4'])
df['log_sales5'] = np.log10(df['sales5'])

# Part 1: Regression 

## Product 1: Tropicana Premium 64 oz

In [20]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales1 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp1 +
                                        feat1""",
                                        data = df)


# step 2: fit the model based on the data
results_1 = lm_best.fit()

# step 3: analyze the summary output
print(results_1.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales1   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     59.46
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           3.56e-34
Time:                        12:39:06   Log-Likelihood:                 57.156
No. Observations:                 116   AIC:                            -98.31
Df Residuals:                     108   BIC:                            -76.28
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4345      0.482      2.977      0.0

In [21]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best = smf.ols(formula =  """log_sales1 ~ log_price1 +
                                        log_price3 +
                                        disp1 +
                                        feat1""",
                                        data = df)


# step 2: fit the model based on the data
results_1_opt = lm_best.fit()

# step 3: analyze the summary output
print(results_1.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales1   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     59.46
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           3.56e-34
Time:                        12:39:07   Log-Likelihood:                 57.156
No. Observations:                 116   AIC:                            -98.31
Df Residuals:                     108   BIC:                            -76.28
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4345      0.482      2.977      0.0

## Product 2: Tropicana Premium 96 oz

In [22]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales2 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp2 +
                                        feat2""",
                                        data = df)


# step 2: fit the model based on the data
results_2 = lm_best.fit()

# step 3: analyze the summary output
print(results_2.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales2   R-squared:                       0.558
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     19.51
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           1.15e-16
Time:                        12:39:07   Log-Likelihood:                 108.06
No. Observations:                 116   AIC:                            -200.1
Df Residuals:                     108   BIC:                            -178.1
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2913      0.317      7.225      0.0

In [23]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best = smf.ols(formula =  """log_sales2 ~ log_price1 +
                                        log_price2 +
                                        disp2 +
                                        feat2""",
                                        data = df)


# step 2: fit the model based on the data
results_2_opt = lm_best.fit()

# step 3: analyze the summary output
print(results_2.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales2   R-squared:                       0.558
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     19.51
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           1.15e-16
Time:                        12:39:07   Log-Likelihood:                 108.06
No. Observations:                 116   AIC:                            -200.1
Df Residuals:                     108   BIC:                            -178.1
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2913      0.317      7.225      0.0

## Product 3: Topicana 64 oz

In [24]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales3 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp3 +
                                        feat3""",
                                        data = df)


# step 2: fit the model based on the data
results_3 = lm_best.fit()

# step 3: analyze the summary output
print(results_3.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales3   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     38.20
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           1.85e-26
Time:                        12:39:08   Log-Likelihood:                -30.148
No. Observations:                 116   AIC:                             76.30
Df Residuals:                     108   BIC:                             98.32
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3313      1.077      2.164      0.0

In [25]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best = smf.ols(formula =  """log_sales3 ~ log_price1 +
                                        log_price3 +
                                        log_price4 +
                                        disp3 +
                                        feat3""",
                                        data = df)


# step 2: fit the model based on the data
results_3_opt = lm_best.fit()

# step 3: analyze the summary output
print(results_3.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales3   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     38.20
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           1.85e-26
Time:                        12:39:08   Log-Likelihood:                -30.148
No. Observations:                 116   AIC:                             76.30
Df Residuals:                     108   BIC:                             98.32
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3313      1.077      2.164      0.0

## Product 4: Minute Maid 64 oz

In [26]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales4 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp4 +
                                        feat4""",
                                        data = df)


# step 2: fit the model based on the data
results_4 = lm_best.fit()

# step 3: analyze the summary output
print(results_4.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales4   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     46.70
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           7.43e-30
Time:                        12:39:08   Log-Likelihood:                 22.112
No. Observations:                 116   AIC:                            -28.22
Df Residuals:                     108   BIC:                            -6.194
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.7515      0.709      3.879      0.0

In [27]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best = smf.ols(formula =  """log_sales4 ~ log_price1 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp4 +
                                        feat4""",
                                        data = df)


# step 2: fit the model based on the data
results_4_opt = lm_best.fit()

# step 3: analyze the summary output
print(results_4.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales4   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     46.70
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           7.43e-30
Time:                        12:39:09   Log-Likelihood:                 22.112
No. Observations:                 116   AIC:                            -28.22
Df Residuals:                     108   BIC:                            -6.194
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.7515      0.709      3.879      0.0

## Product 5: Dominick's 64 oz

In [28]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales5 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp5 +
                                        feat5""",
                                        data = df)


# step 2: fit the model based on the data
results_5 = lm_best.fit()

# step 3: analyze the summary output
print(results_5.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales5   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     30.94
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           4.05e-23
Time:                        12:39:10   Log-Likelihood:                -21.400
No. Observations:                 116   AIC:                             58.80
Df Residuals:                     108   BIC:                             80.83
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2585      0.941     -0.275      0.7

In [31]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best_opt = smf.ols(formula =  """log_sales5 ~ log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp5 +
                                        feat5""",
                                        data = df)


# step 2: fit the model based on the data
results_5_opt = lm_best_opt.fit()

# step 3: analyze the summary output
print(results_5_opt.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales5   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     36.39
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           6.66e-24
Time:                        12:39:19   Log-Likelihood:                -21.446
No. Observations:                 116   AIC:                             56.89
Df Residuals:                     109   BIC:                             76.17
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2921      0.930     -0.314      0.7

# Part 2: Monte Carlo

In [39]:
# Setup constant variables

# set up "known" parameters based on information given about the SKUs:
oz_1 = 64
oz_2 = 96
oz_3 = 64
oz_4 = 64
oz_5 = 64

# set up "known" parameters based historical averages (without log transformation):
grmar1_avg = df['grmar1'].mean()
grmar2_avg = df['grmar2'].mean()
grmar3_avg = df['grmar3'].mean()
grmar4_avg = df['grmar4'].mean()
grmar5_avg = df['grmar5'].mean()

# set up "known" parameters based on log of historical averages:
price1_avg_log = np.log10(df['price1'].mean())
price2_avg_log = np.log10(df['price2'].mean())
price3_avg_log = np.log10(df['price3'].mean())
price4_avg_log = np.log10(df['price4'].mean())
price5_avg_log = np.log10(df['price5'].mean())



##############################################################################
# set up random draw parameters for coefficients - SKU1

# intercept 
intercept_mu_1    = results_1_opt.params[0]  # coefficient from regression
intercept_sigma_1 = results_1_opt.bse[0]     # standard error from regression

# price SKU1
B1_mu_P1_1    = results_1_opt.params[1]      # coefficient from regression
B1_sigma_P1_1 = results_1_opt.bse[1]         # standard error from regression

# price SKU3
B2_mu_P3_1    = results_1_opt.params[2]      # coefficient from regression
B2_sigma_P3_1 = results_1_opt.bse[2]         # standard error from regression

# disp1 SKU1
B3_mu_D1_1 = results_1_opt.params[3]         # coefficient from regression

# feat1 SKU1
B4_mu_F1_1 = results_1_opt.params[4]         # coefficient from regression


##############################################################################
# set up random draw parameters for coefficients - SKU2

# intercept 
intercept_mu_2    = results_2_opt.params[0] # coefficient from regression
intercept_sigma_2 = results_2_opt.bse[0]    # standard error from regression

# price SKU1
B1_mu_P1_2    = results_2_opt.params[1]     # coefficient from regression
B1_sigma_P1_2 = results_2_opt.bse[1]        # standard error from regression

# price SKU2
B2_mu_P2_2    = results_2_opt.params[2]     # coefficient from regression
B2_sigma_P2_2 = results_2_opt.bse[2]        # standard error from regression

# disp2 SKU2
B3_mu_D2_2 = results_2_opt.params[3]        # coefficient from regression

# feat2 SKU2
B4_mu_F2_2 = results_2_opt.params[4]        # coefficient from regression


##############################################################################
# set up random draw parameters for coefficients - SKU3

# intercept 
intercept_mu_3    = results_3_opt.params[0]  # coefficient from regression
intercept_sigma_3 = results_3_opt.bse[0]     # standard error from regression

# price SKU1
B1_mu_P1_3    = results_3_opt.params[1]      # coefficient from regression
B1_sigma_P1_3 = results_3_opt.bse[1]         # standard error from regression

# price SKU3 
B2_mu_P3_3    = results_3_opt.params[2]      # coefficient from regression
B2_sigma_P3_3 = results_3_opt.bse[2]         # standard error from regression

# feat2 SKU4
B3_mu_P4_3    = results_3_opt.params[3]      # coefficient from regression
B3_sigma_P4_3 = results_3_opt.bse[3]         # standard error from regression

# disp3 SKU3
B4_mu_D3_3 = results_3_opt.params[4]         # coefficient from regression

# feat3 SKU3
B5_mu_F3_3 = results_3_opt.params[5]         # coefficient from regression


##############################################################################
# set up random draw parameters for coefficients - SKU4

# intercept 
intercept_mu_4    = results_4_opt.params[0]  # coefficient from regression
intercept_sigma_4 = results_4_opt.bse[0]     # standard error from regression

# price SKU1
B1_mu_P1_4    = results_4_opt.params[1]      # coefficient from regression
B1_sigma_P1_4 = results_4_opt.bse[1]         # standard error from regression

# price SKU3
B2_mu_P3_4    = results_4_opt.params[2]      # coefficient from regression
B2_sigma_P3_4 = results_4_opt.bse[2]         # standard error from regression

# price  SKU4
B3_mu_P4_4    = results_4_opt.params[3]      # coefficient from regression
B3_sigma_P4_4 = results_4_opt.bse[3]         # standard error from regression

# price SKU5
B4_mu_P5_4    = results_4_opt.params[4]      # coefficient from regression
B4_sigma_P5_4 = results_4_opt.bse[4]         # standard error from regression

# disp4 SKU4
B5_mu_D4_4 = results_4_opt.params[5]         # coefficient from regression

# feat4 SKU4
B6_mu_F4_4 = results_4_opt.params[6]         # coefficient from regression


##############################################################################
# set up random draw parameters for coefficients - SKU5 

# intercept 
intercept_mu_5    = results_5_opt.params[0]  # coefficient from regression
intercept_sigma_5 = results_5_opt.bse[0]     # standard error from regression

# price SKU2
B1_mu_P2_5    = results_5_opt.params[1]      # coefficient from regression
B1_sigma_P2_5 = results_5_opt.bse[1]         # standard error from regression

# price SKU3
B2_mu_P3_5    = results_5_opt.params[2]      # coefficient from regression
B2_sigma_P3_5 = results_5_opt.bse[2]         # standard error from regression

# price SKU4
B3_mu_P4_5    = results_5_opt.params[3]      # coefficient from regression
B3_sigma_P4_5 = results_5.bse[3]             # standard error from regression

# price SKU5
B4_mu_P5_5    = results_5_opt.params[4]      # coefficient from regression
B4_sigma_P5_5 = results_5_opt.bse[4]         # standard error from regression

# disp SKU5
B5_mu_D5_5    = results_5_opt.params[5]      # coefficient from regression

# feat5 SKU5
B6_mu_F5_5    = results_5_opt.params[6]      # coefficient from regression

In [41]:
# Simulation

# set up Simulation parameters
np.random.seed(42)  # fix the random seed for reproducibility 

number_sims = 1_000

# Store results from each simulation here
p5_stats = []

for P_5 in arange(0.01546875, 0.03890625,0.001):

    # Begin Simulation Loop
    for i in range(number_sims):
        
        all_stats = [] 
        
        
        ###########
        #  SKU1   # 
        ###########
        
        intercept_1 = np.random.normal(intercept_mu_1, intercept_sigma_1)
        B1_1   = np.random.normal(B1_mu_P1_1, B1_sigma_P1_1)
        B2_1   = np.random.normal(B2_mu_P3_1, B2_sigma_P3_1)   
        feat1  = np.random.randint(2)
        disp1  = np.random.randint(2)
        
        sales_1 = np.exp(intercept_1 \
                + B1_1 * price1_avg_log \
                + B2_1 * price3_avg_log \
                + B3_mu_D1_1 * disp1 \
                + B4_mu_F1_1 * feat1)
        revenue_1 = sales_1 * oz_1 * price1_avg_log
        profits_1 = revenue_1 * grmar1_avg
        
        
        ###########
        #  SKU2   # 
        ###########
        
        intercept_2 = np.random.normal(intercept_mu_2, intercept_sigma_2)
        B1_2   = np.random.normal(B1_mu_P1_2, B1_sigma_P1_2)
        B2_2   = np.random.normal(B2_mu_P2_2, B2_sigma_P2_2)   
        feat2  = np.random.randint(2)
        disp2  = np.random.randint(2)
        
        sales_2 = np.exp(intercept_2 \
                + B1_2 * price1_avg_log \
                + B2_2 * price2_avg_log \
                + B3_mu_D2_2 * disp2 \
                + B4_mu_F2_2 * feat2)
        
        revenue_2 = sales_2 * oz_2 * price2_avg_log
        profits_2 = revenue_2 * grmar2_avg
        
                
        ###########
        #  SKU3   # 
        ###########
        
        intercept_3 = np.random.normal(intercept_mu_3, intercept_sigma_3)
        B1_3   = np.random.normal(B1_mu_P1_3, B1_sigma_P1_3)
        B2_3   = np.random.normal(B2_mu_P3_3, B2_sigma_P3_3)   
        B3_3   = np.random.normal(B3_mu_P4_3, B3_sigma_P4_3) 
        feat3  = np.random.randint(2)
        disp3  = np.random.randint(2)
        
        sales_3 = np.exp(intercept_3 \
                + B1_3 * price1_avg_log \
                + B2_3 * price3_avg_log \
                + B3_3 * price4_avg_log \
                + B4_mu_D3_3 * disp3 \
                + B5_mu_F3_3 * feat3)
        
        revenue_3 = sales_3 * oz_3 * price3_avg_log
        profits_3 = revenue_3 * grmar3_avg
        
        
        ###########
        #  SKU4   # 
        ###########
        
        intercept_4 = np.random.normal(intercept_mu_4, intercept_sigma_4)
        B1_4   = np.random.normal(B1_mu_P1_4, B1_sigma_P1_4)
        B2_4   = np.random.normal(B2_mu_P3_4, B2_sigma_P3_4)   
        B3_4   = np.random.normal(B3_mu_P4_4, B3_sigma_P4_5) 
        B4_4   = np.random.normal(B4_mu_P5_5, B4_sigma_P5_5)
        feat4  = np.random.randint(2)
        disp4  = np.random.randint(2)
        
        sales_4 = np.exp(intercept_4 \
                + B1_4 * price1_avg_log \
                + B2_4 * price3_avg_log \
                + B3_4 * price4_avg_log \
                + B4_4 * (np.log10(P_5)) \
                + B5_mu_D4_4 * disp5 \
                + B6_mu_F4_4 * feat5)
        
        revenue_4 = sales_4 * oz_4 * price4_avg_log
        profits_4 = revenue_4 * grmar4_avg
        
        
        ###########
        #  SKU5   # 
        ###########
        
        intercept_5 = np.random.normal(intercept_mu_5, intercept_sigma_5)
        B1_5   = np.random.normal(B1_mu_P2_5, B1_sigma_P2_5)
        B2_5   = np.random.normal(B2_mu_P3_5, B2_sigma_P3_5)   
        B3_5   = np.random.normal(B3_mu_P4_5, B3_sigma_P4_5) 
        B4_5   = np.random.normal(B4_mu_P5_5, B4_sigma_P5_5)
        feat5  = np.random.randint(2)
        disp5  = np.random.randint(2)
        
        sales_5 = np.exp(intercept_5 \
                + B1_5 * price2_avg_log \
                + B2_5 * price3_avg_log \
                + B3_5 * price4_avg_log \
                + B4_5 * (np.log10(P_5)) \
                + B5_5 * disp5 \
                + B6_5 * feat5)
        
        revenue_5 = sales_5 * oz_5 * P_5
        profits_5 = revenue_5 * grmar5_avg
        
        
#         all_stats.append([sales_5, revenue_5, profits_5, feat5, disp5])

#     # Save results in the results dataframe
#     results_df = pd.DataFrame.from_records(all_stats, 
#                                            columns=['units_sold_5',
#                                                     'revenue_5',
#                                                     'profits_5',
#                                                     'feat5',
#                                                     'disp5']) 
    
#     # Sa    
#     p5_stats.append([P_5,
#                      results_df['revenue_5'].mean(),
#                      results_df['profits_5'].mean()])

# p5_results_df = pd.DataFrame.from_records(p5_stats, columns=['Price per Oz',
#                                                              'Revenue',
#                                                              'Profits'])

# #
# print(f"""
# Maximum Average Profit: {p5_results_df['Profits'].max()}

# Optimal Price for SKU5: {p5_results_df['Price per Oz']}""")

# # Plot scatterplot
# p5_results_df.plot.scatter(x = "Price per Oz", y = "Profits");